This dataset provides information pertaining to vehicle loan default payments. The goal of this project is to see which features are most correlated with a borrowers' loan defaults in their first equated monthly installement (EMI) as well as figuring out the likelihood (probability) of default based off a model that incoporates those features. 

Before we do any analysis on our dataset, it is important that we clean our data. This portion of the project is a step by step process of how I managed to wrangle the data.  

First we need to import our packages and find the directory where our dataset lies.

In [1]:
# Import necessary packages
import pandas as pd
import os
from datetime import datetime

In [2]:
# Go to directory folder where our dataset exists
path="/Users/student/Desktop/SpringBoardDSCareerTrack/Capstone 2 Datasets/Vehicle Loan Payments/data"
os.chdir(path)
os.getcwd()

'/Users/student/Desktop/SpringBoardDSCareerTrack/Capstone 2 Datasets/Vehicle Loan Payments/data'

Next, we load the dataset into a pandas dataframe and explore its contents. 

In [3]:
# Save csv data as a dataframe: df
df = pd.read_csv('train.csv')
df.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,...,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,...,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


In [4]:
df.info()  # check info to see how many null values 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 41 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   UniqueID                             233154 non-null  int64  
 1   disbursed_amount                     233154 non-null  int64  
 2   asset_cost                           233154 non-null  int64  
 3   ltv                                  233154 non-null  float64
 4   branch_id                            233154 non-null  int64  
 5   supplier_id                          233154 non-null  int64  
 6   manufacturer_id                      233154 non-null  int64  
 7   Current_pincode_ID                   233154 non-null  int64  
 8   Date.of.Birth                        233154 non-null  object 
 9   Employment.Type                      225493 non-null  object 
 10  DisbursalDate                        233154 non-null  object 
 11  State_ID     

In [5]:
# We've also been given descriptions for each feature:
df_descriptions = pd.read_csv('data_dictionary.csv', index_col = False)
print(df_descriptions[['Variable Name', 'Description']])

                          Variable Name  \
0                              UniqueID   
1                          loan_default   
2                      disbursed_amount   
3                            asset_cost   
4                                   ltv   
5                             branch_id   
6                           supplier_id   
7                       manufacturer_id   
8                       Current_pincode   
9                         Date.of.Birth   
10                      Employment.Type   
11                        DisbursalDate   
12                             State_ID   
13                     Employee_code_ID   
14                    MobileNo_Avl_Flag   
15                          Aadhar_flag   
16                             PAN_flag   
17                         VoterID_flag   
18                         Driving_flag   
19                        Passport_flag   
20                    PERFORM_CNS.SCORE   
21        PERFORM_CNS.SCORE.DESCRIPTION   
22         

In [6]:
# Confirm all features which have missing data the percentage of NA: NA values
nas=pd.DataFrame(df.isnull().sum().sort_values(ascending=False)/len(df),columns = ['percent'])
pos = nas['percent'] > 0
nas[pos]      # Employment.Type is the only feature with missing values

,percent
Employment.Type,0.032858


In [7]:
# Check the unique values of 'Employent.Type'
df['Employment.Type'].unique()

array(['Salaried', 'Self employed', nan], dtype=object)

In [8]:
# Figure out what to do with missing value for employment type! 

In [9]:
# Check which features have dtype of object
df_col_list = list(df.columns)

object_list = []

for column in df_col_list:
    if df[column].dtype == object:
        object_list.append(column)
        
print(object_list)

['Date.of.Birth', 'Employment.Type', 'DisbursalDate', 'PERFORM_CNS.SCORE.DESCRIPTION', 'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH']


In [10]:
# Check the unique values for each feature of dtype object to validate date is of same format.
for obj in object_list:
    print('The unique values of feature', obj, 'is' , df[obj].unique())

The unique values of feature Date.of.Birth is ['01-01-84' '31-07-85' '24-08-85' ... '24-03-62' '23-07-66' '08-12-62']
The unique values of feature Employment.Type is ['Salaried' 'Self employed' nan]
The unique values of feature DisbursalDate is ['03-08-18' '26-09-18' '01-08-18' '26-10-18' '19-09-18' '23-09-18'
 '16-09-18' '05-09-18' '29-09-18' '03-09-18' '30-08-18' '31-08-18'
 '22-10-18' '18-09-18' '20-09-18' '06-09-18' '11-10-18' '20-10-18'
 '27-08-18' '08-10-18' '17-09-18' '10-09-18' '18-10-18' '16-08-18'
 '10-10-18' '23-10-18' '30-09-18' '21-09-18' '12-09-18' '21-08-18'
 '14-08-18' '17-10-18' '30-10-18' '06-08-18' '04-10-18' '16-10-18'
 '21-10-18' '15-09-18' '06-10-18' '24-10-18' '28-08-18' '13-09-18'
 '31-10-18' '07-09-18' '24-08-18' '05-10-18' '14-10-18' '13-10-18'
 '11-08-18' '28-09-18' '25-10-18' '29-10-18' '10-08-18' '12-08-18'
 '25-08-18' '26-08-18' '08-08-18' '27-10-18' '18-08-18' '23-08-18'
 '20-08-18' '22-09-18' '24-09-18' '15-10-18' '28-10-18' '14-09-18'
 '04-09-18' '13-08

In [11]:
# If we want to do a time series later on, it'd be useful to have 'Date.of.Birth' and 'DisbursalDate' as datetime objects.

# Currently the year is only given as the last two digits. Since 2018 is latest year in dataset, we will add 19 as prefix if 
# digit is greater than 18. Ex. if date of birth is given as '01-01-84', the function will return '01-01-1984'. 
def fixYear(datecol):
    if int(datecol[-2:]) > 18:
        fixedYear = '19' + datecol[-2:]
        newdatecol = datecol[:-2] + fixedYear
    else:
        newdatecol = datecol[:-2] + '20' + datecol[-2:]
    return newdatecol

# Apply the function to 'Date.of.Birth' and 'DisbursalDate' features.
df['Date.of.Birth'] = df['Date.of.Birth'].apply(fixYear)
df['DisbursalDate'] = df['DisbursalDate'].apply(fixYear)

In [12]:
# Run the cell to see the new date formats. This format can be translated into datetime object. 
print('New format for Date.of.Birth:'); print(df['Date.of.Birth'])
print('\n')
print('New format for DisbursalDate:')
print(df['DisbursalDate'])

New format for Date.of.Birth:
0         01-01-1984
1         31-07-1985
2         24-08-1985
3         30-12-1993
4         09-12-1977
             ...    
233149    01-08-1988
233150    05-12-1988
233151    01-06-1976
233152    26-03-1994
233153    18-02-1984
Name: Date.of.Birth, Length: 233154, dtype: object


New format for DisbursalDate:
0         03-08-2018
1         26-09-2018
2         01-08-2018
3         26-10-2018
4         26-09-2018
             ...    
233149    26-10-2018
233150    23-10-2018
233151    24-10-2018
233152    29-09-2018
233153    27-10-2018
Name: DisbursalDate, Length: 233154, dtype: object


In [13]:
# Let's convert Date.of.Birth and DisbursalDate columns into datetime objects. 

# ONLY RUN THIS CELL ONCE. 

# Create datetime object given time format DD-MM-YR
df['Date.of.Birth'] = df['Date.of.Birth'].apply(lambda x: datetime.strptime(x, '%d-%m-%Y'))
df['DisbursalDate'] = df['DisbursalDate'].apply(lambda x: datetime.strptime(x, '%d-%m-%Y'))

# Verify that 'Date.of.Birth' and 'DisbursalDate' is of type datetime. 
print('Date.of.Birth column is of type:',df['Date.of.Birth'].dtypes)
print('Disbursal column is of type:',df['DisbursalDate'].dtypes)

Date.of.Birth column is of type: datetime64[ns]
Disbursal column is of type: datetime64[ns]


In [14]:
# Print the count of unique values for every column, regardless of data type.

for column in df_col_list:
    print(len(df[column].unique()),'unique values for', column , ':',df[column].unique())
    print('\n')

233154 unique values for UniqueID : [420825 537409 417566 ... 613658 548084 630213]


24565 unique values for disbursed_amount : [50578 47145 53278 ... 61456 51406 34259]


46252 unique values for asset_cost : [ 58400  65550  61360 ...  45233  52965 116009]


6579 unique values for ltv : [89.55 73.23 89.63 ... 33.32 32.53 33.82]


82 unique values for branch_id : [ 67  78  34 130  74  11   5  20  63  48  79   3  42 142  36  16 146 147
  65   9   1 152  29  10  70  19   7  85  61  17   8 153  18 162  68  72
  64   2 160 251 103 104 120 136  77  13 138 135  73 248  15 165  62  76
 105 249 250 255 254  82 158 159 117 202 259 207  35  69  97  43 257 258
 260 111  66 261 101  14 121 217  84 100]


2953 unique values for supplier_id : [22807 22744 17014 ... 23618 22289 24031]


11 unique values for manufacturer_id : [ 45  86  48  51 120  49 145  67 153 156 152]


6698 unique values for Current_pincode_ID : [1441 1502 1497 ... 3586  680  693]


15433 unique values for Date.of.Birth : ['1984-0

71341 unique values for PRI.CURRENT.BALANCE : [     0  27600  72879 ...  13064 201422 390443]


44390 unique values for PRI.SANCTIONED.AMOUNT : [     0  50200  74500 ...  85629 276624 416133]


47909 unique values for PRI.DISBURSED.AMOUNT : [     0  50200  74500 ...  80226 237977 416133]


37 unique values for SEC.NO.OF.ACCTS : [ 0  2  1  3 11  9  5  4  7 19  6 16  8 10 23 13 25 46 14 31 15 20 18 28
 30 38 37 29 12 24 17 34 21 42 22 35 52]


23 unique values for SEC.ACTIVE.ACCTS : [ 0  2  1  3 11  4  9  5  7  6 10 26 22  8 21 14 15 12 20 13 36 17 16]


9 unique values for SEC.OVERDUE.ACCTS : [0 1 4 5 3 2 6 8 7]


3246 unique values for SEC.CURRENT.BALANCE : [      0 1171994 5787530 ... 3618737   17326   41108]


2223 unique values for SEC.SANCTIONED.AMOUNT : [      0 1690000   40000 ... 3953133   64040   49246]


2553 unique values for SEC.DISBURSED.AMOUNT : [      0 1690000     361 ... 3953133   64040   49246]


28067 unique values for PRIMARY.INSTAL.AMT : [    0  1991    31 ... 22090

In [15]:
for column in df_col_list:
    print('% unique for', column, ':', (df[column].nunique()/len(df)) * 100 , '%') 

% unique for UniqueID : 100.0 %
% unique for disbursed_amount : 10.535954776671213 %
% unique for asset_cost : 19.837532274805493 %
% unique for ltv : 2.8217401374199027 %
% unique for branch_id : 0.03516988771369996 %
% unique for supplier_id : 1.2665448587628778 %
% unique for manufacturer_id : 0.004717911766471946 %
% unique for Current_pincode_ID : 2.8727793647117355 %
% unique for Date.of.Birth : 6.61923020836014 %
% unique for Employment.Type : 0.0008578021393585355 %
% unique for DisbursalDate : 0.036027689853058494 %
% unique for State_ID : 0.009435823532943892 %
% unique for Employee_code_ID : 1.4025064978512056 %
% unique for MobileNo_Avl_Flag : 0.00042890106967926776 %
% unique for Aadhar_flag : 0.0008578021393585355 %
% unique for PAN_flag : 0.0008578021393585355 %
% unique for VoterID_flag : 0.0008578021393585355 %
% unique for Driving_flag : 0.0008578021393585355 %
% unique for Passport_flag : 0.0008578021393585355 %
% unique for PERFORM_CNS.SCORE : 0.24576031292622041 %


In [16]:
# Let's look at the summary statistics for numerical variables. 
df.describe()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,State_ID,Employee_code_ID,...,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,loan_default
count,233154.000000,233154.000000,2.331540e+05,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,...,233154.000000,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,233154.000000,233154.000000,233154.000000,233154.000000
mean,535917.573376,54356.993528,7.586507e+04,74.746530,72.936094,19638.635035,69.028054,3396.880247,7.262243,1549.477148,...,0.007244,5.427793e+03,7.295923e+03,7.179998e+03,1.310548e+04,3.232684e+02,0.381833,0.097481,0.206615,0.217071
std,68315.693711,12971.314171,1.894478e+04,11.456636,69.834995,3491.949566,22.141304,2238.147502,4.482230,975.261278,...,0.111079,1.702370e+05,1.831560e+05,1.825925e+05,1.513679e+05,1.555369e+04,0.955107,0.384439,0.706498,0.412252
min,417428.000000,13320.000000,3.700000e+04,10.030000,1.000000,10524.000000,45.000000,1.000000,1.000000,1.000000,...,0.000000,-5.746470e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,476786.250000,47145.000000,6.571700e+04,68.880000,14.000000,16535.000000,48.000000,1511.000000,4.000000,713.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,535978.500000,53803.000000,7.094600e+04,76.800000,61.000000,20333.000000,86.000000,2970.000000,6.000000,1451.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
75%,595039.750000,60413.000000,7.920175e+04,83.670000,130.000000,23000.000000,86.000000,5677.000000,10.000000,2362.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.999000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000
max,671084.000000,990572.000000,1.628992e+06,95.000000,261.000000,24803.000000,156.000000,7345.000000,22.000000,3795.000000,...,8.000000,3.603285e+07,3.000000e+07,3.000000e+07,2.564281e+07,4.170901e+06,35.000000,20.000000,36.000000,1.000000


In [17]:
# Save our cleaned dataset into the folder in our path.
path="/Users/student/Desktop/SpringBoardDSCareerTrack/Capstone 2 Datasets/Vehicle Loan Payments/data"
os.chdir(path)
df.to_csv('Vehicle_Loan_DataWrang.csv',index=False)

In [18]:
# Let's see the results
cleaned_df = pd.read_csv('Vehicle_Loan_DataWrang.csv')
print(cleaned_df.head())
print(cleaned_df.info())

   UniqueID  disbursed_amount  asset_cost    ltv  branch_id  supplier_id  \
0    420825             50578       58400  89.55         67        22807   
1    537409             47145       65550  73.23         67        22807   
2    417566             53278       61360  89.63         67        22807   
3    624493             57513       66113  88.48         67        22807   
4    539055             52378       60300  88.39         67        22807   

   manufacturer_id  Current_pincode_ID Date.of.Birth Employment.Type  ...  \
0               45                1441    1984-01-01        Salaried  ...   
1               45                1502    1985-07-31   Self employed  ...   
2               45                1497    1985-08-24   Self employed  ...   
3               45                1501    1993-12-30   Self employed  ...   
4               45                1495    1977-12-09   Self employed  ...   

  SEC.SANCTIONED.AMOUNT  SEC.DISBURSED.AMOUNT  PRIMARY.INSTAL.AMT  \
0          

We've worked with missing values for 'Employment Type' variable as well as converting date oriented features into datetime objects which could potentially be useful when we do a time series analysis for let's say average time from disbursement date to loan default date. We also looked at unique values for each feature as well as the summary statistics for the numerical features. We are well on our way to start our exploratory analysis. 

Little note*: Saving our date to a csv file will not save the datetime object as a datetime. However, it can simply be converted to a datetime object since we took the preliminary steps of saving it in a datetime format.  